![Logo Universidad Galileo](assets/Logo_Universidad_Galileo.png)

# **Lab 1: Prediction of energy prices using multiple regression**
## Course: Statistical Learning
### Student: Otto Chamo


# Libraries

Python libraries and scripts used to work on this assignment

In [138]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression

import utils as utils

# Constants

In [140]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

sheet_names = ["ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO", "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"]
days_of_each_month = {'ENERO': 31, 'FEBRERO': 28, 'MARZO': 31, 'ABRIL': 30, 'MAYO': 31, 'JUNIO': 30, 'JULIO': 31, 'AGOSTO': 31, 'SEPTIEMBRE': 30, 'OCTUBRE': 31, 'NOVIEMBRE': 30, 'DICIEMBRE': 31}

dataset_excel_file_path = "./POE_2023.xlsx"

month_col = "month"
electricity_price_col = "electricityPrice"
hour_of_day_col = "hourOfDay"
date_col = "date"

hora_inicio_col = "horaInicio"
hora_fin_col = "HoraFin"

axis_col = 0

axis_row = 1

# 1. Loading data

In [141]:
datasetExcel = pd.read_excel(dataset_excel_file_path, sheet_name=sheet_names)

# Pre processing

In [142]:
dataframeAllData: pd.DataFrame = pd.DataFrame(columns=[electricity_price_col, month_col, hour_of_day_col, date_col])

In [143]:

for monthIndex, sheetName in enumerate(sheet_names):
    monthDataFrame: pd.DataFrame = datasetExcel[sheetName]
    monthDataFrame: pd.DataFrame = monthDataFrame.drop(hora_inicio_col, axis=1)
    monthDataFrame: pd.DataFrame = monthDataFrame.drop(hora_fin_col, axis=1)
    amountOfMonthDays = monthDataFrame.columns.size

    currentMonth = monthIndex + 1
    print("currentMonth = ", sheetName)
    print("columns = ", monthDataFrame.columns)
    print("columns = ", monthDataFrame.columns.size)
    for day in range(0, days_of_each_month[sheetName]):
       col = day + 1
       startDay = int(day + 1)
       startDate = datetime.datetime(year = 2023, month=currentMonth, day=startDay)
       pricesFrame: pd.DataFrame = monthDataFrame[col].to_frame(name=electricity_price_col)
       pricesFrame[month_col] = currentMonth
       pricesFrame[hour_of_day_col] = range(len(pricesFrame))
       pricesFrame[date_col] = startDate
       dataframeAllData = pd.concat([dataframeAllData, pricesFrame], ignore_index=True)
       print(dataframeAllData)
    #dataframe[month_col] = months[monthIndex]
    #monthDataFrame[month_col] = monthIndex
    #monthDataFrame["HourOfDay"] = range(len(monthDataFrame))
    #dataframe.fillna(value = 0, inplace=True)
print(dataframeAllData)
dataframeAllData.to_excel('output.xlsx')

currentMonth =  ENERO
columns =  Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype='object')
columns =  31
    electricityPrice month hourOfDay       date
0          74.231420     1         0 2023-01-01
1          74.231420     1         1 2023-01-01
2          59.075180     1         2 2023-01-01
3          59.075180     1         3 2023-01-01
4          80.961495     1         4 2023-01-01
5          75.821884     1         5 2023-01-01
6          81.162299     1         6 2023-01-01
7          81.162299     1         7 2023-01-01
8          81.162299     1         8 2023-01-01
9          81.162299     1         9 2023-01-01
10         81.162299     1        10 2023-01-01
11         81.162299     1        11 2023-01-01
12         76.009940     1        12 2023-01-01
13         76.009940     1        13 2023-01-01
14         76.009940     1        14 2023-01-01
15         76.009940    

## Adding month column and applying ordinal encoding to time ranges

In [144]:
weekdayColumn = "Weekday"

for monthIndex, sheetName in enumerate(sheet_names):
    monthDataFrame = datasetExcel[sheetName]
    
    #dataframe[month_col] = months[monthIndex]
    monthDataFrame[month_col] = monthIndex
    monthDataFrame["HourOfDay"] = range(len(monthDataFrame))
    #dataframe.fillna(value = 0, inplace=True)
    startDay = 1
    month = monthIndex + 1


## 1.1 Join dataframes

Each month is in a separate dataframe but we can merge them into a single one because they share the same structure.

In [145]:
dataframes = list(datasetExcel.values())

allData = pd.concat(dataframes)

## Removing columns HoraInicio and HoraFin

We encoded these in the HourOfDay column.

In [146]:
allData = allData.drop('horaInicio', axis=1)
allData = allData.drop('HoraFin', axis=1)

allData.head()

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,month,HourOfDay
0,74.231420,76.322254,76.279396,76.33143,74.943801,61.021538,73.803751,108.739147,76.274707,108.814156,...,76.925243,76.791841,76.687275,76.832013,39.866142,81.658132,40.734832,76.702537,0,0
1,74.231420,76.322254,76.279396,76.33143,76.210497,61.021538,73.803751,76.263791,76.274707,108.814156,...,76.925243,76.791841,76.687275,76.832013,39.866142,81.658132,40.734832,76.702537,0,1
2,59.075180,76.322254,76.279396,76.33143,76.210497,61.021538,73.803751,76.263791,76.274707,108.814156,...,76.925243,76.791841,76.687275,76.832013,39.866142,81.658132,40.734832,76.702537,0,2
3,59.075180,76.322254,76.279396,76.33143,76.210497,61.021538,73.803751,76.263791,76.274707,108.814156,...,76.925243,76.791841,76.687275,76.832013,81.940061,81.658132,40.734832,76.702537,0,3
4,80.961495,76.322254,76.279396,76.33143,76.210497,61.021538,76.254953,76.263791,76.274707,108.814156,...,76.925243,76.791841,76.687275,76.832013,81.940061,81.658132,81.592168,76.702537,0,4


## 1.2 Checking if there are missing values

We start counting for each column.

In [147]:
missing_values_sum = allData.isnull().sum()


missing_values_sum

1              0
2              0
3              0
4              0
5              0
6              0
7              0
8              0
9              0
10             0
11             0
12             0
13             0
14             0
15             0
16             0
17             0
18             0
19             0
20             0
21             0
22             0
23             0
24             0
25             0
26             0
27             0
28             0
29            24
30            24
31           120
month          0
HourOfDay      0
dtype: int64

We can notice that starting from 29th to 31st there are missing values, this is due to not all months having 31 days.

## 1.3 Visualize relationship between predicting variable and the variable we want to predict.

In [148]:
def linePlot(dataframe: pd.DataFrame, title: str):
    hours = dataframe['HourOfDay']
    plt.figure(figsize=(20, 8))
    for i in range(1, 32):
        plt.bar(dataframe[i], hours, label=f'Dia {i}')

    plt.title(title)
    plt.legend()
    plt.gca().set_yticks(np.arange(0, 24, 1))
    plt.grid(True)
    plt.show()

for i in range(0, 12):
    month = months[i]
    monthToPlot = allData[allData['Month'] == i]
    linePlot(monthToPlot, f'Prices per hour for {month}')


KeyError: 'Month'